In [178]:
import pandas as pd
import numpy as np
import seaborn as sns
import copy

In [179]:
df = pd.read_excel('data/syd_sold_property.xlsx')
df_copy =copy.deepcopy(df)

In [180]:
df.columns

Index(['soldPrice', 'landArea', 'priceSource', 'soldYear', 'propertyPrice',
       'address', 'unitNumber', 'streetNumber', 'suburb', 'postcode', 'lat',
       'lng', 'rooms', 'baths', 'parking', 'landSize', 'landUnit',
       'propertyType', 'schools'],
      dtype='object')

In [181]:
df.head()

,soldPrice,landArea,priceSource,soldYear,propertyPrice,address,unitNumber,streetNumber,suburb,postcode,lat,lng,rooms,baths,parking,landSize,landUnit,propertyType,schools
0,850000,103.0,median,2021,650000,"1301/88-90 George Street, Hornsby NSW 2077",1301,88-90,Hornsby,2077,-33.701475,151.100010,2,2,1,0,m虏,apartmentUnitFlat,"[{'id': '', 'educationLevel': 'combined', 'nam..."
1,2050000,917.0,median,2021,1533000,"289 Peats Ferry Road, Hornsby NSW 2077",NaN,289,Hornsby,2077,-33.690778,151.100745,4,2,2,0,m虏,house,"[{'id': '26', 'educationLevel': 'secondary', '..."
2,1105000,184.0,median,2021,1252000,"11/15 Forbes Street, Hornsby NSW 2077",11,15,Hornsby,2077,-33.706390,151.096585,3,2,1,0,m虏,townhouse,"[{'id': '', 'educationLevel': 'secondary', 'na..."
3,679999,24.0,median,2021,650000,"15/215-217 Peats Ferry Road, Hornsby NSW 2077",15,215-217,Hornsby,2077,-33.697822,151.097488,2,1,1,0,m虏,apartmentUnitFlat,"[{'id': '26', 'educationLevel': 'secondary', '..."
4,686000,103.0,listed,2021,680000,"67/121 Pacific Highway, Hornsby NSW 2077",67,121,Hornsby,2077,-33.706767,151.098459,2,2,1,0,m虏,apartmentUnitFlat,"[{'id': '', 'educationLevel': 'combined', 'nam..."


In [182]:
len(df)

13256

In [183]:
df.drop(df.soldYear[df.soldYear<2021].index, inplace=True)

In [184]:
df.drop(columns=['soldYear'], inplace=True)

# Data Analysis

## combine property type

In [186]:
df.propertyType.value_counts()

house                7418
apartmentUnitFlat    2363
townhouse             372
semiDetached          175
vacantLand            146
villa                 136
duplex                 85
terrace                60
acreageSemiRural       36
studio                 28
newHouseLand           26
blockOfUnits           10
newApartments           9
rural                   5
newLand                 3
developmentSite         2
Name: propertyType, dtype: int64

In [187]:
land = ['vacantLand','newLand','newHouseLand']
apartment = ['apartmentUnitFlat', 'newApartments']
rural = ['rural', 'acreageSemiRural']

In [188]:
df.propertyType.replace(land, 'land', inplace=True)
df.propertyType.replace(apartment, 'apartment', inplace=True)
df.propertyType.replace(rural, 'rural', inplace=True)

In [189]:
df.propertyType.value_counts()

house              7418
apartment          2372
townhouse           372
semiDetached        175
land                175
villa               136
duplex               85
terrace              60
rural                41
studio               28
blockOfUnits         10
developmentSite       2
Name: propertyType, dtype: int64

In [190]:
# remove property type less than 10

In [191]:
df.drop(df.propertyType[df.propertyType.isin(['blockOfUnits', 'developmentSite'])].index, inplace=True)

## remove prices which are too high

In [193]:
df.soldPrice.describe()

count    1.086200e+04
mean     1.483760e+06
std      1.182327e+06
min      3.250000e+04
25%      8.000000e+05
50%      1.171750e+06
75%      1.800000e+06
max      5.915000e+07
Name: soldPrice, dtype: float64

In [194]:
df.soldPrice.quantile(0.05), df.soldPrice.quantile(0.95)

(560000.0, 3398999.999999978)

In [195]:
df_clean = df.drop(df.soldPrice[df.soldPrice > df.soldPrice.quantile(0.95)].index)
df_clean = df.drop(df.soldPrice[df.soldPrice < df.soldPrice.quantile(0.05)].index)

In [197]:
df_clean.soldPrice.describe()

count    1.032700e+04
mean     1.536674e+06
std      1.188768e+06
min      5.600000e+05
25%      8.400000e+05
50%      1.220000e+06
75%      1.850000e+06
max      5.915000e+07
Name: soldPrice, dtype: float64

## Area

In [199]:
df.landArea.quantile(0.05),df.landArea.quantile(0.95)

(78.0, 1075.0)

In [200]:
df_clean = df.drop(df.landArea[df.landArea > df.landArea.quantile(0.95)].index)
df_clean = df.drop(df.landArea[df.landArea < df.landArea.quantile(0.05)].index)

In [201]:
len(df_clean)

10342

## Remove Null

In [205]:
df_input = df_clean.drop(columns=['postcode', 'propertyPrice','priceSource', 'address','unitNumber', 'landUnit', 'streetNumber', 'suburb', 'landSize', 'schools'])

In [212]:
df_input.dropna(inplace=True)

In [151]:
# df_clean.to_excel('data/clean_property.xlsx')

# Data pre-process

In [213]:
df_input

,soldPrice,landArea,lat,lng,rooms,baths,parking,propertyType
0,850000,103.0,-33.701475,151.100010,2,2,1,apartment
1,2050000,917.0,-33.690778,151.100745,4,2,2,house
2,1105000,184.0,-33.706390,151.096585,3,2,1,townhouse
4,686000,103.0,-33.706767,151.098459,2,2,1,apartment
5,603000,99.0,-33.704417,151.095720,2,1,1,apartment
...,...,...,...,...,...,...,...,...
13235,1444000,467.0,-33.942986,151.115630,3,1,2,house
13240,1550000,366.0,-33.944188,151.109836,4,2,2,duplex
13245,1691500,481.0,-33.940322,151.113173,3,1,3,house
13246,670000,128.0,-33.939009,151.114516,2,2,1,apartment


## dummy property type and suburb

In [214]:
cat_list = pd.get_dummies(df_input['propertyType'])

In [215]:
df_input=df_input.join(cat_list).drop(columns=['propertyType'])

In [216]:
df_input.head()

,soldPrice,landArea,lat,lng,rooms,baths,parking,apartment,duplex,house,land,rural,semiDetached,studio,terrace,townhouse,villa
0,850000,103.0,-33.701475,151.100010,2,2,1,1,0,0,0,0,0,0,0,0,0
1,2050000,917.0,-33.690778,151.100745,4,2,2,0,0,1,0,0,0,0,0,0,0
2,1105000,184.0,-33.706390,151.096585,3,2,1,0,0,0,0,0,0,0,0,1,0
4,686000,103.0,-33.706767,151.098459,2,2,1,1,0,0,0,0,0,0,0,0,0
5,603000,99.0,-33.704417,151.095720,2,1,1,1,0,0,0,0,0,0,0,0,0


# Pearson correlation coefficient

In [217]:
df_input.columns

Index(['soldPrice', 'landArea', 'lat', 'lng', 'rooms', 'baths', 'parking',
       'apartment', 'duplex', 'house', 'land', 'rural', 'semiDetached',
       'studio', 'terrace', 'townhouse', 'villa'],
      dtype='object')

In [218]:
var = [ 'landArea', 'lat', 'lng', 'rooms', 'baths', 'parking']

In [219]:
house = df_input.loc[df_input.house[df_input.house==1].index]

In [220]:
res = []
for v in var:
    print(np.corrcoef(house[v], house.soldPrice)[0][1])

0.00952745672283638
0.14244837630375812
0.5300347080678828
0.2157332284698483
0.36797580609773867
0.12032336008668437


# ML

In [221]:
X = df_input.drop(['soldPrice'],axis=1)
y = df_input['soldPrice']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 2021)

In [222]:
df_input

,soldPrice,landArea,lat,lng,rooms,baths,parking,apartment,duplex,house,land,rural,semiDetached,studio,terrace,townhouse,villa
0,850000,103.0,-33.701475,151.100010,2,2,1,1,0,0,0,0,0,0,0,0,0
1,2050000,917.0,-33.690778,151.100745,4,2,2,0,0,1,0,0,0,0,0,0,0
2,1105000,184.0,-33.706390,151.096585,3,2,1,0,0,0,0,0,0,0,0,1,0
4,686000,103.0,-33.706767,151.098459,2,2,1,1,0,0,0,0,0,0,0,0,0
5,603000,99.0,-33.704417,151.095720,2,1,1,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13235,1444000,467.0,-33.942986,151.115630,3,1,2,0,0,1,0,0,0,0,0,0,0
13240,1550000,366.0,-33.944188,151.109836,4,2,2,0,1,0,0,0,0,0,0,0,0
13245,1691500,481.0,-33.940322,151.113173,3,1,3,0,0,1,0,0,0,0,0,0,0
13246,670000,128.0,-33.939009,151.114516,2,2,1,1,0,0,0,0,0,0,0,0,0


In [223]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [258]:
models=[LinearRegression(),RandomForestRegressor(),BaggingRegressor()]
models_str=['LinearRegression','RandomForestRegressor', 'BaggingRegressor']
model_d = {}
for name,model in zip(models_str,models):
    print('Training model：'+name)
    model=model 
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)  
    score = r2_score(y_test,y_pred)
    print(name +' score:'+str(score))
    print('----')
    model_d[name] = model

Training model：LinearRegression
LinearRegression score:0.5193993613667508
----
Training model：RandomForestRegressor
RandomForestRegressor score:0.7608097580940671
----
Training model：BaggingRegressor
BaggingRegressor score:0.6985936104130117
----


## choose RandomForestRegressor() as our model

In [259]:
rf = model_d['RandomForestRegressor']

In [260]:
y_pred = rf.predict(X_test)
mean_squared_error(y_test, y_pred)

265059723354.6655

In [266]:
r2_score(y_test,y_pred)

0.7608097580940671

## save model

In [267]:
import pickle
with open('model/rf.pickle','wb') as f:
     pickle.dump(rf, f)